In [14]:
import random

# Helper to display expressions in nicer logical notation
def display_expression(expr):
    replacements = {
        "(not A) or B": "A → B",
        "(not B) or A": "B → A",
        "A == B": "A ↔ B",
        "A != B": "A ⊕ B",
        "not (A and B)": "¬(A ∧ B)",
        "not (A or B)": "¬(A ∨ B)",
        "A and B": "A ∧ B",
        "A or B": "A ∨ B",
        "(not A) and B": "¬A ∧ B",
        "A and not B": "A ∧ ¬B",
        "(not A) or (not B)": "¬A ∨ ¬B",
        "(not A) and (not B)": "¬A ∧ ¬B",
        "A and not B": "A ∧ ¬B",
        "B and not A": "B ∧ ¬A"
    }
    return replacements.get(expr, expr)

def logical_equivalence_quiz(rounds=10):
    print("🔍 Logical Equivalence Quiz")
    print("Decide whether the two expressions are logically equivalent.")
    print("Type True or False and press Enter.\n")

    expressions = [
        #"A and B",
        #"A or B",
        "(not A) and B",
        "A and not B",
        "not (A and B)",
        "not (A or B)",
        "(not A) or (not B)",
        "(not A) and (not B)",
        "A == B",          
        #"A != B",          
        "(not A) or B",      # A → B
        "(not B) or A",      # B → A
        "A and not B",   # negation of A → B
        "B and not A",   # negation of B → A
    ]

    score = 0
    for i in range(1, rounds + 1):
        expr1 = random.choice(expressions)
        expr2 = random.choice(expressions)
        while expr2 == expr1:
            expr2 = random.choice(expressions)

        # Check logical equivalence by truth table
        equivalent = all(
            eval(expr1, {}, {"A": a, "B": b}) == eval(expr2, {}, {"A": a, "B": b})
            for a in [True, False]
            for b in [True, False]
        )

        print(f"Round {i}:")
        print("  Expression 1:", display_expression(expr1))
        print("  Expression 2:", display_expression(expr2))

        guess = input("Are they logically equivalent? (True/False): ").strip()
        guess_bool = guess.lower().startswith("t")

        if guess_bool == equivalent:
            print("✅ Correct!\n")
            score += 1
        else:
            print(f"❌ Incorrect. The right answer was {equivalent}.\n")

    print(f"🏁 Final score: {score}/{rounds}")
    if score == rounds:
        print("🎉 Perfect! You got them all.")
    elif score >= rounds * 0.7:
        print("👍 Great job! You're close — review the tricky ones.")
    else:
        print("📘 Keep practicing — pay attention to how → and ↔ behave.")


In [15]:
# Run the quiz
logical_equivalence_quiz(rounds=2)


🔍 Logical Equivalence Quiz
Decide whether the two expressions are logically equivalent.
Type True or False and press Enter.

Round 1:
  Expression 1: B ∧ ¬A
  Expression 2: ¬A ∨ ¬B
✅ Correct!

Round 2:
  Expression 1: B ∧ ¬A
  Expression 2: A ↔ B
✅ Correct!

🏁 Final score: 2/2
🎉 Perfect! You got them all.


## Random Arguments

In [12]:
import itertools
import random

# --- 1. Random literal (with at most one 'not') ---
def random_literal(variables):
    var = random.choice(variables)
    return var if random.random() > 0.5 else f"not {var}"

# --- 2. Build a random formula recursively ---
def random_formula(variables, depth=2):
    if depth == 0:
        return random_literal(variables)
    left = random_formula(variables, depth - 1)
    right = random_formula(variables, depth - 1)
    op = random.choice(["and", "or", "->", "<->"])
    if op == "->":
        return f"(not {left} or {right})"  # p -> q ≡ ¬p ∨ q
    elif op == "<->":
        return f"(({left} and {right}) or (not {left} and not {right}))"
    else:
        return f"({left} {op} {right})"

# --- 3. Evaluate a formula for a given row of truth values ---
def eval_formula(formula, values):
    return eval(formula, {}, values)

# --- 4. Build truth table ---
def truth_table(variables, formulas):
    table = []
    for combo in itertools.product([False, True], repeat=len(variables)):
        vals = dict(zip(variables, combo))
        row = {v: vals[v] for v in variables}
        for label, f in formulas.items():
            row[label] = eval_formula(f, vals)
        table.append(row)
    return table

# --- 5. Generate a random argument ---
def random_argument(variables=["p", "q", "r"], num_premises=2):
    premises = [random_formula(variables) for _ in range(num_premises)]
    conclusion = random_formula(variables)
    return premises, conclusion

# --- 6. Check argument validity ---
def check_argument(premises, conclusion, variables):
    formulas = {f"P{i+1}": prem for i, prem in enumerate(premises)}
    formulas["Conclusion"] = conclusion
    table = truth_table(variables, formulas)
    valid = True
    counterexamples = []
    for row in table:
        if all(row[f"P{i+1}"] for i in range(len(premises))) and not row["Conclusion"]:
            valid = False
            counterexamples.append(row)
    return valid, counterexamples

# --- 7. Pretty-print formulas with logical symbols ---
def pretty(formula):
    # Replace carefully: longer substrings first
    formula = formula.replace("<->", "↔")
    formula = formula.replace("->", "→")
    formula = formula.replace(" and ", " ∧ ")
    formula = formula.replace(" or ", " ∨ ")
    formula = formula.replace("not ", "¬")
    return formula


In [15]:

# --- Demo run ---
if __name__ == "__main__":
    premises, conclusion = random_argument()
    print("🧠 Randomly generated argument:")
    for i, p in enumerate(premises, 1):
        print(f"  Premise {i}: {pretty(p)}")
    print(f"  Conclusion: {pretty(conclusion)}")

    valid, counterexamples = check_argument(premises, conclusion, ["p", "q", "r"])

    print("\n📊 Result:")
    if valid:
        print("✅ The argument is VALID (no counterexamples).")
    else:
        print("❌ The argument is INVALID. Counterexamples:")
        for c in counterexamples:
            print("   " + ", ".join(f"{k}={v}" for k, v in c.items()))


🧠 Randomly generated argument:
  Premise 1: (((¬p ∧ ¬p) ∨ (¬¬p ∧ ¬¬p)) ∧ (p ∧ r))
  Premise 2: ((p ∨ ¬q) ∧ ((q ∧ p) ∨ (¬q ∧ ¬p)))
  Conclusion: ((¬r ∨ ¬p) ∧ (r ∧ r))

📊 Result:
❌ The argument is INVALID. Counterexamples:
   p=True, q=True, r=True, P1=True, P2=True, Conclusion=False


In [13]:

# --- Demo run ---
premises, conclusion = random_argument()
print("🧠 Randomly generated argument:")
for i, p in enumerate(premises, 1):
    print(f"  Premise {i}: {pretty(p)}")
print(f"  Conclusion: {pretty(conclusion)}")

valid, counterexamples = check_argument(premises, conclusion, ["p", "q", "r"])

print("\n📊 Result:")
if valid:
    print("✅ The argument is VALID (no counterexamples).")
else:
    print("❌ The argument is INVALID. Counterexamples:")
    for c in counterexamples:
        print("   " + ", ".join(f"{k}={v}" for k, v in c.items()))


🧠 Randomly generated argument:
  Premise 1: (¬(¬¬p ∨ p) ∨ ((¬q ∧ p) ∨ (¬¬q ∧ ¬p)))
  Premise 2: ((¬r ∨ ¬p) ∨ (¬¬q ∨ p))
  Conclusion: ((¬r ∨ p) ∧ (r ∨ ¬q))

📊 Result:
❌ The argument is INVALID. Counterexamples:
   p=False, q=False, r=True, P1=True, P2=True, Conclusion=False
   p=False, q=True, r=False, P1=True, P2=True, Conclusion=False
   p=False, q=True, r=True, P1=True, P2=True, Conclusion=False
